In [1]:
import requests
from bs4 import BeautifulSoup
import threading
import queue
import re

In [2]:
# free-proxy-list.net上面的ip保存在本地ip_pool.txt文件
def ipToFile():
    url = 'https://free-proxy-list.net/'
    r = requests.get(url)
    html = r.text
    soup = BeautifulSoup(html, features='html5lib')
    div_tag = soup.find('div', 'table-responsive')
    tbody_list = div_tag.tbody.contents
    result = ''
    for tr in tbody_list:
        ip = tr.next_element.string
        port = tr.next_element.next_sibling.string
        result += ip + ':' + port + '\n'

    with open('ip_pool.txt', 'w') as f:
        f.write(result)

In [3]:
# 从ip_pool读取ip信息
# 返回queue
def getIPool():
    proxies = queue.Queue()
    
    with open('ip_pool.txt', 'r') as f:
        ip_pool = f.read()
        
    ip_pool = ip_pool.split('\n')

    # 删除最后一个空值
    del ip_pool[-1]

    for ip in ip_pool:
        proxy = {'http':ip, 'https':ip}
        proxies.put(proxy)
        
    return proxies

In [4]:
# 获得清理过后的proxies
# 剔除不能连接网页的proxy
# 接收queue, 线程数
# 返回queue
def getCleanProxies(proxies, thread_num):    
#     url = 'http://www.baidu.com'
    url = 'https://jobs.51job.com/'
    
    # 有效的ip队列
    ok_proxies = queue.Queue()
    
    
    def check_proxy(thread_proxies):
        
        thread_proxies_size = thread_proxies.qsize()
        
        for i in range(thread_proxies_size):
            thread_proxies_size = thread_proxies.qsize()
            print('当前剩余处理的ip: ', thread_proxies_size)
            
            # 如果当前ip为0退出当前线程
            if(thread_proxies_size == 0):
                break
                
            proxy = thread_proxies.get()
            try:
                r = requests.get(url, proxies=proxy, timeout=5)
                print(threading.current_thread().name + ' is processing')
                print('r.status_code:', r.status_code)
                if(r.status_code != 200):
                    print('remove proxy:', proxy)
                else:
                    ok_proxies.put(proxy)

                print('-------------------------------------')

            except requests.Timeout as e:
                print(threading.current_thread().name + ' is processing')
                print(e)
                print('remove proxy:', proxy)
                print('-------------------------------------')
            except requests.ConnectionError as e:
                print(threading.current_thread().name + ' is processing')
                print(e)
                print('remove proxy:', proxy)
                print('-------------------------------------')
            except requests.InvalidHeader as e:
                print(threading.current_thread().name + ' is processing')
                print(e)
                print('remove proxy:', proxy)
                print('-------------------------------------')
      
    
    threads = []
    
    for i in range(thread_num):
        thread = threading.Thread(target=check_proxy, args=(proxies,))
        threads.append(thread)
    
    print('线程开始：')
    for thread in threads:
        print(thread.name + '开始运行')
        thread.start()
    
    # 每次在剩余处理ip为0之后，程序就会停住不动
    # 当当前线程检测到剩余ip为0后，调用join()方法
    print('线程开始停止运行：')
    for thread in threads:
        thread.join()
        print(thread.name + '已停止')
        
    return ok_proxies

----

In [5]:
#  获取可用的ip
ipToFile()
proxies = getIPool()
proxies = getCleanProxies(proxies, 25)
print('可用的ip: ', proxies.qsize())

线程开始：
Thread-4开始运行
当前剩余处理的ip: Thread-5开始运行
 300
当前剩余处理的ip: Thread-6开始运行
 299
当前剩余处理的ip: Thread-7开始运行
 298
当前剩余处理的ip: Thread-8开始运行
 297
当前剩余处理的ip: Thread-9开始运行
 296
当前剩余处理的ip: Thread-10开始运行
 295
当前剩余处理的ip: Thread-11开始运行
 294
当前剩余处理的ip: Thread-12开始运行
 293
当前剩余处理的ip: Thread-13开始运行
 292
当前剩余处理的ip: Thread-14开始运行
 291
当前剩余处理的ip: Thread-15开始运行
 290
当前剩余处理的ip: Thread-16开始运行
 289
当前剩余处理的ip: Thread-17开始运行
 288
当前剩余处理的ip: Thread-18开始运行
 287
当前剩余处理的ip: Thread-19开始运行
 286
当前剩余处理的ip: Thread-20开始运行
 285
当前剩余处理的ip: Thread-21开始运行
 284
当前剩余处理的ip: Thread-22开始运行
 283
当前剩余处理的ip: Thread-23开始运行
 282
当前剩余处理的ip: Thread-24开始运行
 281
当前剩余处理的ip: Thread-25开始运行
 280
当前剩余处理的ip: Thread-26开始运行
 279
当前剩余处理的ip: Thread-27开始运行
 278
当前剩余处理的ip: Thread-28开始运行
 277
Thread-21 is processing
HTTPSConnectionPool(host='jobs.51job.com', port=443): Max retries exceeded with url: / (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 500 Internal Server Error',)))
remove proxy: {'http': '108.61.186.207:

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/home/everglow/anaconda3/lib/python3.6/site-packages/urllib3/response.py", line 331, in _error_catcher
    yield
  File "/home/everglow/anaconda3/lib/python3.6/site-packages/urllib3/response.py", line 640, in read_chunked
    chunk = self._handle_chunk(amt)
  File "/home/everglow/anaconda3/lib/python3.6/site-packages/urllib3/response.py", line 586, in _handle_chunk
    value = self._fp._safe_read(amt)
  File "/home/everglow/anaconda3/lib/python3.6/http/client.py", line 614, in _safe_read
    raise IncompleteRead(b''.join(s), amt)
http.client.IncompleteRead: IncompleteRead(9881 bytes read, 359 more expected)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/everglow/anaconda3/lib/python3.6/site-packages/requests/models.py", line 749, in generate
    for chunk in self.raw.stream(chunk_size, decode_content=True):
  File "/home/everglow/anaconda3/lib

Thread-13 is processing
r.status_code: 200
-------------------------------------
当前剩余处理的ip:  14
Thread-6 is processing
HTTPSConnectionPool(host='jobs.51job.com', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f3be83f3a90>, 'Connection to 124.108.17.9 timed out. (connect timeout=5)'))
remove proxy: {'http': '124.108.17.9:8080', 'https': '124.108.17.9:8080'}
-------------------------------------
当前剩余处理的ip:  13
Thread-17 is processing
HTTPSConnectionPool(host='jobs.51job.com', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f3be843b4e0>, 'Connection to 186.226.162.156 timed out. (connect timeout=5)'))
remove proxy: {'http': '186.226.162.156:8080', 'https': '186.226.162.156:8080'}
-------------------------------------
当前剩余处理的ip:  12
Thread-19 is processing
HTTPSConnectionPool(host='jobs.51job.com', port=443): Max retries ex

In [6]:
# 爬取单页的数据
url = 'https://search.51job.com/list/000000,000000,0000,00,9,99,Java,2,1.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare='
proxy = proxies.get()
r = requests.get(url, proxies=proxy)
r.encoding = 'gbk'
html = r.text
soup = BeautifulSoup(html, features='html5lib')
#找到了工作列表的开头
title = soup.find('div', 'el title')
# 找到每一个工作列，它们分别是一个div class='el'
job_info_list = title.find_next_siblings('div', 'el')

In [7]:
class jobObject:
    'job描述对象'
    __slots__ = ('tag_href', 'job', 'company', 'address', 'salary', 'tag')

In [8]:
# 存放当前页jobObject对象
jobObjectList = []
for tag in job_info_list:
    object = jobObject()
    object.tag_href = tag.find('a').attrs['href']
    object.job = tag.find('a').attrs['title']
    
    infos = tag.find_all(class_=re.compile('t[2-4]'))
    object.company = infos[0].string
    object.address = infos[1].string
    object.salary = infos[2].string
    jobObjectList.append(object)

print('组装完毕')

print('共有', len(jobObjectList), '个工作')

组装完毕
共有 50 个工作


In [9]:
num = 1
for i in jobObjectList:
    print(num)
    print(i.tag_href)
    print(i.job)
    print(i.company)
    print(i.address)
    print(i.salary)
    print()
    num += 1

1
https://jobs.51job.com/guangzhou-thq/103873674.html?s=01&t=0
Java高级开发工程师(卖家平台研发)
广州江楠鲜品科技有限公司
广州-天河区
1.5-2.5万/月

2
https://jobs.51job.com/guangzhou/82879764.html?s=01&t=0
JAVA开发工程师
广州众攀网络科技有限公司
广州
0.6-1万/月

3
https://jobs.51job.com/guangzhou-thq/98702320.html?s=01&t=0
游戏JAVA服务端
广州雷麒网络科技有限公司
广州-天河区
0.5-1万/月

4
https://jobs.51job.com/hefei-ssq/102440784.html?s=01&t=0
零基础JAVA开发+高新上岗
合肥贤丰资产管理有限公司
合肥-蜀山区
0.8-1万/月

5
https://jobs.51job.com/beijing-cyq/104520709.html?s=01&t=0
Java高级开发工程师
北京百里半网络技术有限公司
北京-朝阳区
1.2-2.4万/月

6
https://jobs.51job.com/shantou/99461330.html?s=01&t=0
Java开发工程师
广东东升信息技术有限公司
汕头
4-8千/月

7
https://jobs.51job.com/wuhan-hsq/94539550.html?s=01&t=0
Java开发讲师/助教
武汉尚观信息科技有限公司
武汉-洪山区
0.8-1.2万/月

8
https://jobs.51job.com/wuhan-dhxjs/98572237.html?s=01&t=0
Java开发工程师
武汉怡图信息技术有限公司
武汉-东湖新技...
0.5-1.2万/月

9
https://jobs.51job.com/fuzhou-glq/104517897.html?s=01&t=0
Java后端开发
福州人资通信息技术有限公司
福州-鼓楼区
1-1.5万/月

10
https://jobs.51job.com/beijing-cyq/104521520.html?s=01&t=0
JAVA工程师(J10557)
北京河

In [10]:
# 用一个队列来存储详情链接
tag_href_queue = queue.Queue()

for i in jobObjectList:
    print(i.tag_href)
    tag_href_queue.put(i.tag_href)

print('队列中的tag_href: ', tag_href_queue.qsize())

https://jobs.51job.com/guangzhou-thq/103873674.html?s=01&t=0
https://jobs.51job.com/guangzhou/82879764.html?s=01&t=0
https://jobs.51job.com/guangzhou-thq/98702320.html?s=01&t=0
https://jobs.51job.com/hefei-ssq/102440784.html?s=01&t=0
https://jobs.51job.com/beijing-cyq/104520709.html?s=01&t=0
https://jobs.51job.com/shantou/99461330.html?s=01&t=0
https://jobs.51job.com/wuhan-hsq/94539550.html?s=01&t=0
https://jobs.51job.com/wuhan-dhxjs/98572237.html?s=01&t=0
https://jobs.51job.com/fuzhou-glq/104517897.html?s=01&t=0
https://jobs.51job.com/beijing-cyq/104521520.html?s=01&t=0
https://jobs.51job.com/guangzhou-byq/98162683.html?s=01&t=0
https://jobs.51job.com/nanchang-hgt/102261768.html?s=01&t=0
https://jobs.51job.com/hangzhou-xhq/104520766.html?s=01&t=0
https://jobs.51job.com/shenzhen-lhxq/103833741.html?s=01&t=0
https://jobs.51job.com/shenzhen/104520229.html?s=01&t=0
https://jobs.51job.com/fuzhou-tjq/102854972.html?s=01&t=0
https://jobs.51job.com/foshan-ccq/101350925.html?s=01&t=0
https://j

In [11]:
# 获取详情页标签
# 返回一个包含有标签的list
# 接收详情页的链接， ip代理
def getTag(url, proxies):
    tag_url = url
    proxy = proxies.get()
    
    # 存储标签的List
    tag_list = []
    flag = True
    
    #TODO 查看信息
    print('getTag()当前处理链接:', tag_url)
    print('getTag()获得的ip:', proxy)
    while flag:
    #     headers = {'user-agent','Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/66.0.3359.181 Chrome/66.0.3359.181 Safari/537.36'}
        try:
            r_tag = requests.get(tag_url, timeout=5, proxies=proxy)
        
        # 报了异常的话捕捉了之后
        # 获得新的proxy之后就直接开始下次循环
        except requests.exceptions.Timeout as e:
            print(e)
            proxy = proxies.get()
            print('----------------------')
            continue
        except requests.exceptions.ProxyError as e:
            print(e)
            proxy = proxies.get()
            print('----------------------')
            continue
        except requests.exceptions.ConnectionError as e:
            print(e)
            proxy = proxies.get()
            print('----------------------')
            continue
        
        # 不报异常就取消循环
        flag = False
            
        r_tag.encoding = 'gbk'
        tag_html = r_tag.text
        tag_soup = BeautifulSoup(tag_html, features='html5lib')

        #找到详情页第一行标签
        tag_1 = tag_soup.find_all('span', 'sp4')
        for i in tag_1:
            tag_list.append(i.contents[1])

        #找到详情页第二行标签
        tag_2 = tag_soup.find('p', 't2')

        #有些详情页没有第二行标签
        if tag_2 is None:
            print('----------------------')
            return tag_list

        ll = []
        for i in tag_2.strings:
            ll.append(i)

        #清除ll中的空值
        def not_empty(s):
            return s and s.strip()

        for i in filter(not_empty, ll):
            tag_list.append(i)
        
    print('getTag()中tag_list:', tag_list)
    print('----------------------')
    
    return tag_list

In [12]:
for i in range(tag_href_queue.qsize()):
    jobObjectList[i].tag = getTag(tag_href_queue.get(), proxies)
    
print('tag标签组装完成！')

getTag()当前处理链接: https://jobs.51job.com/guangzhou-thq/103873674.html?s=01&t=0
getTag()获得的ip: {'http': '182.53.204.39:53281', 'https': '182.53.204.39:53281'}
getTag()中tag_list: ['3-4年经验', '本科', '招2人', '07-14发布', '五险一金', '绩效奖金', '弹性工作', '年终奖金', '餐饮补贴']
----------------------
getTag()当前处理链接: https://jobs.51job.com/guangzhou/82879764.html?s=01&t=0
getTag()获得的ip: {'http': '182.253.100.136:3129', 'https': '182.253.100.136:3129'}
HTTPSConnectionPool(host='jobs.51job.com', port=443): Max retries exceeded with url: /guangzhou/82879764.html?s=01&t=0 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f3be83e3470>, 'Connection to 182.253.100.136 timed out. (connect timeout=5)'))
----------------------
getTag()中tag_list: ['无工作经验', '招3人', '07-14发布', '员工旅游', '股票期权', '弹性工作', '五险', '周末双休', '年终奖金', '定期体检', '带薪年假', '带薪病假', '下午茶']
----------------------
getTag()当前处理链接: https://jobs.51job.com/guangzhou-thq/98702320.html?s=01&t=0
getTag()获得的ip: {'http': '136.24.133.49:522

In [13]:
num = 1
for i in jobObjectList:
    print(num)
    print(i.tag_href)
    print(i.job)
    print(i.company)
    print(i.address)
    print(i.salary)
    print(i.tag)
    num += 1

1
https://jobs.51job.com/guangzhou-thq/103873674.html?s=01&t=0
Java高级开发工程师(卖家平台研发)
广州江楠鲜品科技有限公司
广州-天河区
1.5-2.5万/月
['3-4年经验', '本科', '招2人', '07-14发布', '五险一金', '绩效奖金', '弹性工作', '年终奖金', '餐饮补贴']
2
https://jobs.51job.com/guangzhou/82879764.html?s=01&t=0
JAVA开发工程师
广州众攀网络科技有限公司
广州
0.6-1万/月
['无工作经验', '招3人', '07-14发布', '员工旅游', '股票期权', '弹性工作', '五险', '周末双休', '年终奖金', '定期体检', '带薪年假', '带薪病假', '下午茶']
3
https://jobs.51job.com/guangzhou-thq/98702320.html?s=01&t=0
游戏JAVA服务端
广州雷麒网络科技有限公司
广州-天河区
0.5-1万/月
['1年经验', '大专', '招若干人', '07-14发布', '五险一金', '绩效奖金', '弹性工作', '不加班', '节日礼物', '年终奖金', '股票期权', '定期体检', '专业培训']
4
https://jobs.51job.com/hefei-ssq/102440784.html?s=01&t=0
零基础JAVA开发+高新上岗
合肥贤丰资产管理有限公司
合肥-蜀山区
0.8-1万/月
['无工作经验', '中专', '招8人', '07-14发布', '五险一金', '员工旅游', '交通补贴', '餐饮补贴', '专业培训', '出国机会', '绩效奖金', '年终奖金', '弹性工作', '定期体检']
5
https://jobs.51job.com/beijing-cyq/104520709.html?s=01&t=0
Java高级开发工程师
北京百里半网络技术有限公司
北京-朝阳区
1.2-2.4万/月
['5-7年经验', '本科', '招10人', '07-14发布']
6
https://jobs.51job.com/shantou/99461330.html?s=

In [18]:
import csv

In [24]:
headers = ['职位', '公司', '地址', '薪酬', '标签']

datas = []

# fd = {headers[4]: jobObjectList[0].tag}
# fd

for jobObject in jobObjectList:
    data = {headers[0]:jobObject.job,
            headers[1]:jobObject.company, 
            headers[2]:jobObject.address,
            headers[3]:jobObject.salary,
            headers[4]:jobObject.tag}
    datas.append(data)

In [25]:
with open('some.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, headers)
    writer.writeheader()
    for row in datas:
        writer.writerow(row)